# Importacion de Librerias

In [1]:
import os
import requests
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from tfx.components import CsvExampleGen
from tfx.components import SchemaGen, Transform, StatisticsGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from ml_metadata.proto import metadata_store_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration.metadata import sqlite_metadata_connection_config
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_data_validation as tfdv
import glob
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.components import ExampleValidator
from tfx.components import Trainer
from tfx.components import Transform
from tfx.proto import trainer_pb2
from tfx.components import Evaluator


2024-03-04 03:13:14.391199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 03:13:15.907644: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-04 03:13:15.907844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

# 2.1. Cargar el Dataset

In [2]:
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [3]:
ORIGINAL_DATASET_PATH = 'data/covertype/covertype_train.csv'
df_full = pd.read_csv(ORIGINAL_DATASET_PATH)
print(df_full.shape)
df_full.head()

(116203, 13)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


# 3 Selección de características

Para implementar ingeniería de características y seleccionar las mejores K variables usando SelectKBest de scikit-learn, es necesario primero preparar los datos separando las variables independientes (características) de la variable dependiente (objetivo). Luego, se aplicará SelectKBest para seleccionar las K mejores características basadas en la prueba de ANOVA F-value para clasificación, que es lo que *f_classif* realiza.

Ahora bien, dado que se cuenta con variables categoricas, generamos una duminifación de las mismas.
### Codificación One-Hot


In [4]:
df_full = pd.get_dummies(df_full, drop_first=True)

## Paso 1: 
### Preparar los Datos
Identificar cuál es la variable objetivo la cual, para el ejemplo es *Cover_Type*, que indica el tipo de cobertura forestal. 

In [5]:
# Paso 1: Preparar los datos
X = df_full.drop('Cover_Type', axis=1) # Características
y = df_full['Cover_Type'] # Variable objetivo

## Paso 3: 
### Partición de datos en entrenamiento y prueba
Se genera la partición de datos con un 30% de prueba y 70% de entrenamiento, tanto escalados como normales

In [6]:
#Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


## Paso 4: 
### Aplicar SelectKBest
A continuación, se aplicará SelectKBest para seleccionar las K mejores características (en este caso serán 6), se aplica el SelectKBest posterior a la partición, dado que evitaría el *Data Leakage* asegurando que el proceso de selección no esté influenciado por información del conjunto de prueba. Esto ayuda a prevenir el data leakage, donde la información del conjunto de prueba "se filtra" al modelo durante la fase de entrenamiento, llevando a estimaciones excesivamente optimistas del rendimiento del modelo.
Por otro lado, ayudaría a la validación más realista, dado que al realizar la selección de características solo en el conjunto de entrenamiento, se simula mejor el escenario real donde el modelo se enfrenta a datos no vistos, proporcionando una evaluación más realista de su capacidad de generalización:

In [7]:
#Aplicar SelectKBest
select = SelectKBest(score_func=f_classif, k=10)  # Ajusta k según el número deseado de características
X_train_selected_raw = select.fit_transform(X_train, y_train)
X_test_selected_raw = select.transform(X_test)
mask = select.get_support()
# Extraer los nombres de las columnas seleccionadas basado en la máscara
selected_columns = X.columns[mask]
print(selected_columns)

Index(['Elevation', 'Horizontal_Distance_To_Roadways', 'Wilderness_Area_Rawah',
       'Soil_Type_C2703', 'Soil_Type_C2704', 'Soil_Type_C2705',
       'Soil_Type_C2717', 'Soil_Type_C4703', 'Soil_Type_C8771',
       'Soil_Type_C8772'],
      dtype='object')


In [8]:
# Crear DataFrames para los conjuntos de entrenamiento y prueba con las características seleccionadas
df_train = pd.DataFrame(X_train_selected_raw, columns=selected_columns)
df_test = pd.DataFrame(X_test_selected_raw, columns=selected_columns)

# Agregar la variable objetivo a los DataFrames
df_train['Cover_Type'] = y_train.reset_index(drop=True)
df_test['Cover_Type'] = y_test.reset_index(drop=True)
df_train.head()

,Elevation,Horizontal_Distance_To_Roadways,Wilderness_Area_Rawah,Soil_Type_C2703,Soil_Type_C2704,Soil_Type_C2705,Soil_Type_C2717,Soil_Type_C4703,Soil_Type_C8771,Soil_Type_C8772,Cover_Type
0,2884,5210,1,0,0,0,0,0,0,0,1
1,3008,443,0,0,0,0,0,0,0,0,1
2,2955,5251,1,0,0,0,0,0,0,0,0
3,3087,2337,0,0,0,0,0,0,0,0,0
4,3256,6109,1,0,0,0,0,0,0,0,0


# 4 Data Pipeline
## 4.1 Configurar el contexto interactivo

### Configurar el Directorio de Trabajo
Creación de un directorio en tu sistema donde guardarás los datos de tu proyecto, incluyendo la base de datos SQLite para los metadatos de ML.

In [9]:
import sqlite3
conn = sqlite3.connect('covertype.db')
df_train.to_sql('df_train', conn, if_exists='replace', index=False)
df_test.to_sql('df_test', conn, if_exists='replace', index=False)
conn.close()

In [10]:
# Rutas al directorio donde se guardaran los metadatos y otros datos
workspace_dir = os.path.join(os.getcwd(), 'tfx_workspace')
metadata_path = os.path.join(workspace_dir, 'metadata', 'metadata.db')

# Creación del directorio si no existe
os.makedirs(os.path.dirname(metadata_path), exist_ok=True)


# Configurar el Contexto Interactivo
Para trabajar con TFX interactivamente, especialmente en Jupyter Notebooks, necesitamos configurar un contexto interactivo. Este contexto se encargará de gestionar los metadatos y permitirte ejecutar componentes de TFX individualmente.

In [11]:
context = InteractiveContext(
    pipeline_name='interactive_pipeline',
    pipeline_root=workspace_dir,
    metadata_connection_config={
        'sqlite_filename': metadata_path
    }
)

# Usar el Contexto Interactivo
## Configurar InteractiveContext
Primero, posteior a importar ConnectionConfig desde ml_metadata.proto, se debe creae un ConnectionConfig para la base de datos SQLite y úsarlo para configurar el InteractiveContext:
Una vez configurado el contexto interactivo, se puede comenzar a definir y ejecutar componentes de TFX directamente en el Notebook:

Este enfoque creará un ConnectionConfig que especifica la ruta a tu archivo de base de datos SQLite y el modo de conexión. Usar filename_uri para especificar la ubicación de tu archivo de base de datos y connection_mode para indicar cómo debe abrirse la base de datos.

In [12]:
# Configuración del contexto interactivo con la ruta a la base de datos de metadatos
context = InteractiveContext(
    pipeline_name='interactive_pipeline',
    pipeline_root=workspace_dir,
    metadata_connection_config=sqlite_metadata_connection_config(metadata_path)
)

Cargar datos al entorno de tfx

In [13]:
# Ruta completa donde deseas guardar los archivos CSV



import csv
csv_file = 'tfx_workspace/data/covertype_train.csv'
conn = sqlite3.connect('covertype.db') 
cursor = conn.cursor()
query = "SELECT * FROM df_train"
cursor.execute(query)
with open(csv_file, 'w', newline='') as file:         
    csv_writer = csv.writer(file)         # Opcional: Escribir los nombres de las columnas como la primera fila en el CSV        
    csv_writer.writerow([i[0] for i in cursor.description])         # Escribir los datos        
    csv_writer.writerows(cursor)
cursor.close()    
conn.close()



csv_file = 'tfx_workspace/data/covertype_test.csv'
conn = sqlite3.connect('covertype.db') 
cursor = conn.cursor()
query = "SELECT * FROM df_test"
cursor.execute(query)
with open(csv_file, 'w', newline='') as file:         
    csv_writer = csv.writer(file)         # Opcional: Escribir los nombres de las columnas como la primera fila en el CSV        
    csv_writer.writerow([i[0] for i in cursor.description])         # Escribir los datos        
    csv_writer.writerows(cursor)
cursor.close()    
conn.close()


ruta_df_train = 'tfx_workspace/data/covertype_train.csv'
ruta_df_test = 'tfx_workspace/data/covertype_test.csv'



# Imprimir confirmación
print("Archivos .csv guardados en las siguientes rutas:")
print("Ruta del DataFrame de entrenamiento:", ruta_df_train)
print("Ruta del DataFrame de prueba:", ruta_df_test)


Archivos .csv guardados en las siguientes rutas:
Ruta del DataFrame de entrenamiento: tfx_workspace/data/covertype_train.csv
Ruta del DataFrame de prueba: tfx_workspace/data/covertype_test.csv


## 4.2. Generando Ejemplos

In [14]:
import shutil
try:
    files_in_directory = os.listdir('tfx_workspace/data/')
    hidden_files = [file for file in files_in_directory if file.startswith('.')]
    for file in hidden_files:
        file_path = os.path.join('tfx_workspace/data/', file)
        shutil.rmtree('tfx_workspace/data/')
        print(f'Archivo oculto eliminado: {file_path}')
    data_dir = os.path.join(workspace_dir, 'data')
except:
    print("no hidden directories")
example_gen = CsvExampleGen(input_base=data_dir)
# Ejecutando CsvExampleGen
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 196
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [15]:
from tfx.components import CsvExampleGen

# Crear un CsvExampleGen para el conjunto de entrenamiento
train_example_gen = CsvExampleGen(input_base='tfx_workspace/data', input_config={
    "splits": [
        {"name": "train", "pattern": "covertype_train.csv"}
    ]
})
context.run(train_example_gen)

# Crear un CsvExampleGen para el conjunto de prueba
eval_example_gen = CsvExampleGen(input_base='tfx_workspace/data', input_config={
    "splits": [
        {"name": "test", "pattern": "covertype_test.csv"}
    ]
})
context.run(eval_example_gen)


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 198
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## 4.3. Estadísticas

In [16]:
# Calcular estadísticas para el conjunto de entrenamiento
train_statistics_gen = StatisticsGen(examples=train_example_gen.outputs['examples'])
context.run(train_statistics_gen)

# Calcular estadísticas para el conjunto de prueba
eval_statistics_gen = StatisticsGen(examples=eval_example_gen.outputs['examples'])
context.run(eval_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 200
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

## 4.4 Inferir el esquema

In [17]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)
context.show(statistics_gen.outputs['statistics'])

In [18]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

# Visualizar el esquema inferido
context.show(schema_gen.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Soil_Type_C2703',INT,required,,-
'Soil_Type_C2704',INT,required,,-
'Soil_Type_C2705',INT,required,,-
'Soil_Type_C2717',INT,required,,-
'Soil_Type_C4703',INT,required,,-
'Soil_Type_C8771',INT,required,,-


In [19]:
def encontrar_id_mas_reciente(base_path):
    # Construcción de la ruta completa al directorio de estadísticas
    search_path = os.path.join(base_path, 'StatisticsGen', 'statistics', '*')
    
    # Encuentración de todos los subdirectorios en la ruta base
    subdirs = glob.glob(search_path)
    
    # Ordenamiento de los subdirectorios por fecha de modificación, el más reciente primero
    subdirs.sort(key=os.path.getmtime, reverse=True)
    
    # Extrae el ID más reciente del primer subdirectorio en la lista
    if subdirs:
        latest_id = os.path.basename(subdirs[0])
        return latest_id
    else:
        raise Exception("No se encontraron subdirectorios de estadísticas.")
    
# Uso de la función para obtener el ID más reciente
base_path = 'tfx_workspace'
latest_id = encontrar_id_mas_reciente(base_path)
print("ID más reciente:", latest_id)


ID más reciente: 201


In [20]:
# Construcción de las rutas a los archivos FeatureStats.pb para cada split utilizando el ID más reciente
train_stats_path = os.path.join(base_path, 'StatisticsGen', 'statistics', latest_id, 'Split-train', 'FeatureStats.pb')
eval_stats_path = os.path.join(base_path, 'StatisticsGen', 'statistics', latest_id, 'Split-eval', 'FeatureStats.pb')

print("Ruta a las estadísticas de entrenamiento:", train_stats_path)
print("Ruta a las estadísticas de evaluación:", eval_stats_path)

# Cargar las estadísticas
train_stats = tfdv.load_stats_binary(train_stats_path)
eval_stats = tfdv.load_stats_binary(eval_stats_path)
# Visualizar y comparar las estadísticas

tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats, lhs_name='EVAL', rhs_name='TRAIN')

Ruta a las estadísticas de entrenamiento: tfx_workspace/StatisticsGen/statistics/201/Split-train/FeatureStats.pb
Ruta a las estadísticas de evaluación: tfx_workspace/StatisticsGen/statistics/201/Split-eval/FeatureStats.pb


## 4.5. Curando el esquema

# Cargar el Esquema Inferido
Primero, cargaremos el esquema inferido que generaste previamente con SchemaGen.

In [21]:
def encontrar_id_mas_reciente(base_path):
    # Construye la ruta completa al directorio del generador de esquemas (SchemaGen)
    search_path = os.path.join(base_path, 'SchemaGen', 'schema', '*')
    
    # Encuentra todos los archivos de esquemas en la ruta base
    schema_files = glob.glob(search_path)
    
    # Ordena los archivos de esquemas por fecha de modificación, el más reciente primero
    schema_files.sort(key=os.path.getmtime, reverse=True)
    
    # Extrae el nombre del archivo del esquema más reciente
    if schema_files:
        latest_schema_file = os.path.basename(schema_files[0])
        return latest_schema_file
    else:
        raise Exception("No se encontraron archivos de esquemas en el directorio SchemaGen.")

# Uso de la función para obtener el nombre del esquema más reciente
base_path = 'tfx_workspace'
latest_schema_file = encontrar_id_mas_reciente(base_path)
print("Nombre del esquema más reciente:", latest_schema_file)

schema_path = os.path.join(base_path, 'SchemaGen', 'schema', latest_schema_file, 'schema.pbtxt')
# Carga el esquema
schema = tfdv.load_schema_text(schema_path)


Nombre del esquema más reciente: 202


In [22]:
adjusted_schema_path = os.path.join('tfx_workspace', 'SchemaGen', 'adjusted_schema', 'schema.pbtxt')
os.makedirs(os.path.dirname(adjusted_schema_path), exist_ok=True)

tfdv.set_domain(schema, 'Elevation', schema_pb2.IntDomain(name='Elevation', min=1860, max=3858))
tfdv.set_domain(schema, 'Horizontal_Distance_To_Roadways', schema_pb2.IntDomain(name='Horizontal_Distance_To_Roadways', min=0, max=7116))
tfdv.set_domain(schema, 'Soil_Type_C2703', schema_pb2.IntDomain(name='Soil_Type_C2703', min=0, max=1,is_categorical=True))
tfdv.set_domain(schema, 'Soil_Type_C2704', schema_pb2.IntDomain(name='Soil_Type_C2704', min=0, max=1,is_categorical=True))
tfdv.set_domain(schema, 'Soil_Type_C2705', schema_pb2.IntDomain(name='Soil_Type_C2705', min=0, max=1,is_categorical=True))
tfdv.set_domain(schema, 'Soil_Type_C2717', schema_pb2.IntDomain(name='Soil_Type_C2717', min=0, max=1,is_categorical=True))
tfdv.set_domain(schema, 'Soil_Type_C4703', schema_pb2.IntDomain(name='Soil_Type_C4703', min=0, max=1,is_categorical=True))
tfdv.set_domain(schema, 'Soil_Type_C8771', schema_pb2.IntDomain(name='Soil_Type_C8771', min=0, max=1,is_categorical=True))
tfdv.set_domain(schema, 'Soil_Type_C8772', schema_pb2.IntDomain(name='Soil_Type_C8772', min=0, max=1,is_categorical=True))
tfdv.set_domain(schema, 'Wilderness_Area_Rawah', schema_pb2.IntDomain(name='Wilderness_Area_Rawah', min=0, max=1))
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))
# Guardar el esquema ajustado

tfdv.write_schema_text(schema, adjusted_schema_path)

In [23]:
adjusted_schema = tfdv.load_schema_text(adjusted_schema_path)
tfdv.display_schema(schema=adjusted_schema)


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,min: 1860; max: 3858
'Horizontal_Distance_To_Roadways',INT,required,,min: 0; max: 7116
'Soil_Type_C2703',INT,required,,min: 0; max: 1
'Soil_Type_C2704',INT,required,,min: 0; max: 1
'Soil_Type_C2705',INT,required,,min: 0; max: 1
'Soil_Type_C2717',INT,required,,min: 0; max: 1
'Soil_Type_C4703',INT,required,,min: 0; max: 1
'Soil_Type_C8771',INT,required,,min: 0; max: 1


In [24]:
cover_type_domain = tfdv.get_domain(schema, 'Cover_Type')
if cover_type_domain and getattr(cover_type_domain, 'is_categorical', False):
    print('Cover_Type es categórica.')
else:
    print('Cover_Type no está configurada como categórica o no existe en el esquema.')

Cover_Type es categórica.


## Utilizar el Esquema Ajustado en el Pipeline TFX

In [25]:
# Asumiendo que `statistics_gen` es tu componente StatisticsGen y `schema_gen` es tu componente SchemaGen
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 203
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

## 4.6. Entornos de esquema
### Configurar Entornos de Esquema

In [26]:
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_data_validation as tfdv

adjusted_schema = tfdv.load_schema_text(adjusted_schema_path)

TRAINING_ENVIRONMENT = 'TRAINING'
SERVING_ENVIRONMENT = 'SERVING'

if TRAINING_ENVIRONMENT not in adjusted_schema.default_environment:
    adjusted_schema.default_environment.append(TRAINING_ENVIRONMENT)
if SERVING_ENVIRONMENT not in adjusted_schema.default_environment:
    adjusted_schema.default_environment.append(SERVING_ENVIRONMENT)

for feature in adjusted_schema.feature:
    if feature.name == 'Cover_Type':
        # Asegúrate de que 'Cover_Type' no sea incluida en el entorno de SERVING
        if SERVING_ENVIRONMENT not in feature.not_in_environment:
            feature.not_in_environment.append(SERVING_ENVIRONMENT)

modified_schema_path = 'modified_schema.pbtxt'
tfdv.write_schema_text(adjusted_schema, modified_schema_path)
print(f"El esquema modificado, con 'Cover_Type' eliminado del entorno de SERVING, se ha guardado en: {modified_schema_path}")









El esquema modificado, con 'Cover_Type' eliminado del entorno de SERVING, se ha guardado en: modified_schema.pbtxt


## 4.7. Anomalias del ambiente train

In [27]:
statistics = tfdv.generate_statistics_from_dataframe(df_train)
adjusted_schema = tfdv.load_schema_text(modified_schema_path)
TRAINING_ENVIRONMENT = 'TRAINING'
adjusted_schema.default_environment[:] = [TRAINING_ENVIRONMENT]
anomalies = tfdv.validate_statistics(statistics, adjusted_schema, environment=TRAINING_ENVIRONMENT)
tfdv.display_anomalies(anomalies)
df_train.head()

,Elevation,Horizontal_Distance_To_Roadways,Wilderness_Area_Rawah,Soil_Type_C2703,Soil_Type_C2704,Soil_Type_C2705,Soil_Type_C2717,Soil_Type_C4703,Soil_Type_C8771,Soil_Type_C8772,Cover_Type
0,2884,5210,1,0,0,0,0,0,0,0,1
1,3008,443,0,0,0,0,0,0,0,0,1
2,2955,5251,1,0,0,0,0,0,0,0,0
3,3087,2337,0,0,0,0,0,0,0,0,0
4,3256,6109,1,0,0,0,0,0,0,0,0


## 4.8. Comprobar anomalías
### Prueba

## Validar los Conjuntos de Datos de Servicio
### Entrenamiento

In [28]:
frac = 0.5
df_service = df_train.copy()
df_service = df_train.sample(frac=frac, random_state=42, ignore_index= True)
df_service = df_service.drop(columns = ['Cover_Type'])
df_service = df_service.reset_index(drop=True)
statistics = tfdv.generate_statistics_from_dataframe(df_service)
adjusted_schema = tfdv.load_schema_text(modified_schema_path)
SERVING_ENVIRONMENT = 'SERVING'
adjusted_schema.default_environment[:] = [SERVING_ENVIRONMENT]
anomalies = tfdv.validate_statistics(statistics, adjusted_schema, environment=SERVING_ENVIRONMENT)
tfdv.display_anomalies(anomalies)
df_service.head()

,Elevation,Horizontal_Distance_To_Roadways,Wilderness_Area_Rawah,Soil_Type_C2703,Soil_Type_C2704,Soil_Type_C2705,Soil_Type_C2717,Soil_Type_C4703,Soil_Type_C8771,Soil_Type_C8772
0,3150,395,1,0,0,0,0,0,0,0
1,3030,1409,1,0,0,0,0,0,0,0
2,2813,859,0,0,0,0,0,0,0,0
3,2896,5255,1,0,0,0,0,0,0,0
4,2856,3510,1,0,0,0,0,0,0,0


## 4.9. Ingeniería de características
### Definir la función de preprocesamiento

In [29]:
from tfx.components import Transform 
transform = Transform( examples=example_gen.outputs['examples'], 
                      schema=schema_gen.outputs['schema'],
                      preprocessing_fn='preprocessing.py')



In [81]:
print(transform.outputs)

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}), 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}), 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}), 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}), 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}), 'post_transform_schema': OutputChannel(artifact_

# 5. Metadatos de aprendizaje automático

In [30]:

from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'tfx_workspace/metadata/metadata.db'
connection_config.sqlite.connection_mode = 3  
store = metadata_store.MetadataStore(connection_config)


## 5.1. Acceso a artefactos almacenados

In [31]:
artifact_types = store.get_artifact_types()
for artifact_type in artifact_types:
    print(artifact_type.name)


Examples
ExampleStatistics
Schema
TransformGraph
TransformCache
ExampleAnomalies
Model
ModelRun


In [32]:
artifact_types = store.get_artifact_types()
for artifact_type in artifact_types:
    print(artifact_type.name)

Examples
ExampleStatistics
Schema
TransformGraph
TransformCache
ExampleAnomalies
Model
ModelRun


In [66]:
artifacts = store.get_artifacts_by_type('Examples')
for artifact in artifacts:
    print(artifact.id, artifact.uri)


1 /tfx/src/tfx_workspace/CsvExampleGen/examples/1
2 /tfx/src/tfx_workspace/CsvExampleGen/examples/2
3 /tfx/src/tfx_workspace/CsvExampleGen/examples/3
4 ./tfx_workspace/CsvExampleGen/examples/4
5 ./tfx_workspace/CsvExampleGen/examples/5
6 /tfx/src/tfx_workspace/CsvExampleGen/examples/6
7 /tfx/src/tfx_workspace/CsvExampleGen/examples/7
8 /tfx/src/tfx_workspace/CsvExampleGen/examples/8
9 /tfx/src/tfx_workspace/CsvExampleGen/examples/9
10 /tfx/src/tfx_workspace/CsvExampleGen/examples/10
11 /tfx/src/tfx_workspace/CsvExampleGen/examples/11
12 /tfx/src/tfx_workspace/CsvExampleGen/examples/12
15 /tfx/src/tfx_workspace/CsvExampleGen/examples/15
16 /tfx/src/tfx_workspace/CsvExampleGen/examples/16
17 /tfx/src/tfx_workspace/CsvExampleGen/examples/17
18 /tfx/src/tfx_workspace/CsvExampleGen/examples/18
19 /tfx/src/tfx_workspace/CsvExampleGen/examples/19
20 /tfx/src/tfx_workspace/CsvExampleGen/examples/20
24 /tfx/src/tfx_workspace/Transform/transformed_examples/23
32 /tfx/src/tfx_workspace/Transform/

In [67]:
schema_artifacts = store.get_artifacts_by_type('Schema')
latest_schema = max(schema_artifacts, key=lambda a: a.id)
print(latest_schema.uri)

/tfx/src/tfx_workspace/SchemaGen/schema/202


## 5.2. Seguimiento de artefactos

In [35]:
artifact_types = store.get_artifact_types()
for artifact_type in artifact_types:
    print(artifact_type.name)

Examples
ExampleStatistics
Schema
TransformGraph
TransformCache
ExampleAnomalies
Model
ModelRun


In [68]:
def find_parent_artifacts(store, artifact_id):
    parents = []
    events = store.get_events_by_artifact_ids([artifact_id])
    for event in events:
        if event.type == metadata_store_pb2.Event.OUTPUT:
            execution_id = event.execution_id
            input_events = store.get_events_by_execution_ids([execution_id])
            for input_event in input_events:
                if input_event.type == metadata_store_pb2.Event.INPUT:
                    parent_artifact_id = input_event.artifact_id
                    parent_artifact = store.get_artifacts_by_id([parent_artifact_id])[0]
                    parents.append(parent_artifact)
    return parents

In [69]:
transform_graph_artifacts = store.get_artifacts_by_type('TransformGraph')
for artifact in transform_graph_artifacts:
    print(artifact.id, artifact.uri)


23 /tfx/src/tfx_workspace/Transform/transform_graph/23
31 /tfx/src/tfx_workspace/Transform/transform_graph/25
39 /tfx/src/tfx_workspace/Transform/transform_graph/28
101 /tfx/src/tfx_workspace/Transform/transform_graph/97
117 /tfx/src/tfx_workspace/Transform/transform_graph/106
139 /tfx/src/tfx_workspace/Transform/transform_graph/118
209 /tfx/src/tfx_workspace/Transform/transform_graph/177
225 /tfx/src/tfx_workspace/Transform/transform_graph/186
233 /tfx/src/tfx_workspace/Transform/transform_graph/187
241 /tfx/src/tfx_workspace/Transform/transform_graph/188
249 /tfx/src/tfx_workspace/Transform/transform_graph/189
257 /tfx/src/tfx_workspace/Transform/transform_graph/190
265 /tfx/src/tfx_workspace/Transform/transform_graph/191
273 /tfx/src/tfx_workspace/Transform/transform_graph/192
281 /tfx/src/tfx_workspace/Transform/transform_graph/193
289 /tfx/src/tfx_workspace/Transform/transform_graph/194
297 /tfx/src/tfx_workspace/Transform/transform_graph/195


In [70]:
ARTIFACT_ID = 97
parent_artifacts = find_parent_artifacts(store, ARTIFACT_ID)

print("Artefactos padres del TransformGraph con ID", ARTIFACT_ID)
for parent_artifact in parent_artifacts:
    print(f"ID: {parent_artifact.id}, Type: {parent_artifact.type_id}, URI: {parent_artifact.uri}")



Artefactos padres del TransformGraph con ID 97
ID: 95, Type: 14, URI: /tfx/src/tfx_workspace/CsvExampleGen/examples/90


## 5.3. Obtener artefactos principales

In [71]:
ARTIFACT_ID = 97  
parent_artifacts = find_parent_artifacts(store, ARTIFACT_ID)
print("Artefactos padres para el artefacto ID:", ARTIFACT_ID)
for parent_artifact in parent_artifacts:
    artifact_type = store.get_artifact_types_by_id([parent_artifact.type_id])[0].name
    print(f"Artifact ID: {parent_artifact.id}, Type: {artifact_type}, URI: {parent_artifact.uri}")


Artefactos padres para el artefacto ID: 97
Artifact ID: 95, Type: Examples, URI: /tfx/src/tfx_workspace/CsvExampleGen/examples/90


In [75]:

def find_executions_for_artifact(artifact_id):
    events = store.get_events_by_artifact_ids([artifact_id])
    execution_ids = [event.execution_id for event in events if event.type == metadata_store_pb2.Event.OUTPUT]
    return execution_ids

def find_input_artifacts_for_executions(execution_ids):
    events = store.get_events_by_execution_ids(execution_ids)
    input_artifact_ids = [event.artifact_id for event in events if event.type == metadata_store_pb2.Event.INPUT]
    return store.get_artifacts_by_id(input_artifact_ids)

artifact_id = 249
execution_ids = find_executions_for_artifact(artifact_id)
print("Ejecuciones que generaron el artefacto", artifact_id, ":", execution_ids)

input_artifacts = find_input_artifacts_for_executions(execution_ids)
print("Artefactos utilizados como entrada para las ejecuciones:", execution_ids)
for artifact in input_artifacts:
    print(f"ID: {artifact.id}, Tipo: {artifact.type_id}, URI: {artifact.uri}")


Ejecuciones que generaron el artefacto 249 : [189]
Artefactos utilizados como entrada para las ejecuciones: [189]
ID: 223, Tipo: 18, URI: /tfx/src/tfx_workspace/SchemaGen/schema/184


____________________